In [1]:
"""
Tshikana Rasehlomi
e-mail: rasehlomi@gmail.com
DS_CODE CHALLENGE
QUESTION 2: Time Series Challenge - Predict expected weekly service requests
10-07-2022
"""

# Load required modules
#==================================================================================
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# read and load the sr_hex.csv data and check the dataframe columns
#=========================================================================================
sr_hex_df = pd.read_csv('https://cct-ds-code-challenge-input-data.s3.af-south-1.amazonaws.com/sr_hex.csv.gz', compression='gzip', header=0, sep=',', quotechar='\"')
print(sr_hex_df.columns)

In [ ]:
#  we extract input dataframe to be used in our model development by dropping unwanted columns and check the first rows of new dataframe
#======================================================================================================================================
sr_hex_mod = sr_hex_df.drop(['reference_number', 'completion_timestamp',
       'directorate', 'department', 'branch', 'section', 'latitude', 'creation_timestamp',
       'longitude', 'h3_level8_index'], axis=1)
sr_hex_mod.reset_index(inplace=True)
print(sr_hex_mod.head())

In [ ]:
# we then convert categorical fields to numerical values to be used in model development
#=======================================================================================================================
number = LabelEncoder()
sr_hex_mod['code'] = number.fit_transform(sr_hex_mod['code'].astype('str'))
sr_hex_mod['cause_code'] = number.fit_transform(sr_hex_mod['cause_code'].astype('str'))
sr_hex_mod['code_group'] = number.fit_transform(sr_hex_mod['code_group'].astype('str'))
sr_hex_mod['official_suburb'] = number.fit_transform(sr_hex_mod['official_suburb'].astype('str'))
sr_hex_mod['notification_number'] = number.fit_transform(sr_hex_mod['notification_number'].astype('str'))
sr_hex_mod['cause_code_group'] = number.fit_transform(sr_hex_mod['cause_code_group'].astype('str'))
print(sr_hex_mod.head())

In [ ]:
# we define a way to handle the official suburb in the datasets
#=====================================================================================================
def compute_service(x):
    accept = [range(0,8)]
    if x not in accept:
        return 0
    else:
        return x
    
sr_hex_mod['official_suburb'] = sr_hex_mod['official_suburb'].apply(compute_service)

In [ ]:
# we define independent and dependent variables from our data prepared for model development
#========================================================================================================
y = sr_hex_mod['code'].values
X = sr_hex_mod.values

In [ ]:
# Split data into training and test samples of 80%-20%
#==========================================================================================================
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state=0)

In [ ]:
# we compute a RandomForest classifier by providing number of trees (n_estimators) and
# minimum number of samples required to be at a leaf node for the tree (min_samples_leaf)
#==============================================================================================================
Rf = RandomForestClassifier(n_estimators=20, min_samples_leaf=5, random_state=0)
Rf.fit(X_train, y_train)
y_pred = Rf.predict(X_test)
print(y_pred)

In [ ]:
# check the difference between predicted vs actual service requests
#====================================================================================================================

diff =pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
print(diff.head())

# let's visualize these difference on a graph
#=====================================================================================================================

plt.figure(figsize=(8, 8))
ax = sns.distplot(y, hist=False, color="r", label="Actual Value")
sns.distplot(y_pred, hist=False, color="b", label="Fitted Values" , ax=ax)
plt.title('Actual vs Fitted Values for service requests')
plt.show()
plt.close()

In [ ]:
# evaluate model performance and once happy use it to predict the service request per hex for the next 4 weeks
#==============================================================================================================
print('Accuracy: ', accuracy_score(y_test, y_pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
# our model achieved 99% accuracy which is greater than the initial target of 95%
# we confidently use it to predicted expected service requests per hex for next 4 weeks (roughly 28 days)
# using our model we make a one-step prediction

y_pred = Rf.predict(X_test)

expected_service_requests = y_pred[28]

print(expected_service_requests)